# Subsetting GEDI L4A Example

## 1.Define Area of Interest

If your AOI is a publicly available geoBoundary, see
[Getting the GeoJSON URL for a geoBoundary](#getting-the-geojson-url-for-a-geoboundary) \
for details on obtaining it's URL.  In this case, that is the URL you must
supply for the `aoi` input.

Alternatively, you can make your own GeoJSON file for your AOI and place it \
within your public bucket within the ADE.  Based upon where you place your \
GeoJSON file, you can construct a URL to specify for the a job's `aoi` input \
where `path/to/aio.geojson` can be any path and filename for your \
AOI within `my-public-bucket`.


You would then supply the following URL as the `aoi` input value when running \
this algorithm as a DPS job, where `<USERNAME>` is your ADE username and where \
`path/to/aio.geojson` is the location of your GeoJSON file within `my-public-bucket`:


```plain
    aoi = "https://maap-ops-workspace.s3.amazonaws.com/shared/<USERNAME>/path/to/aoi.geojson"
```


In [ ]:
# (Optional Cell) Verify the contents of the area of interest
import geopandas as gpd
import contextily as ctx

aoi = gpd.read_file('/projects/shared-buckets/<USERNAME>/path/to/aoi.geojson.geojson')
aoi_epsg4326 = aoi.to_crs(epsg=4326) # Convert if not already
ax=aoi_epsg4326.plot(figsize=(10, 5), alpha=0.3, edgecolor='red')
ctx.add_basemap(ax, crs=4326)

## 2. Submit a Job

When supplying input values, to use the default value (for the fields below), enter a dash (`-`) as the input value.

<hr>

aoi (required): URL[str]: URL to a GeoJSON file representing your area of interest. \
See [above](#1.Define-Area-of-Interest) for specifics.

columns: Sequence[str]: Comma-separated list of column names to include in output file. \
(Default: agbd, agbd_se, l2_quality_flag, l4_quality_flag, sensitivity, sensitivity_a2)

query: str: Query expression for subsetting the rows in the output file. \
<b>IMPORTANT</b>: The columns input must contain at least all of the columns that appear \
in this query expression, otherwise an error will occur. \
(Default: l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95 and sensitivity_a2 > 0.95")

limit: Maximum number of GEDI granule data files to download (among those that intersect the specified AOI). (Default: 10,000)

In [ ]:
# Imports and Default values
import uuid
from maap.maap import MAAP

maap = MAAP(maap_host='api.ops.maap-project.org')

# submitJob parameters
_identifier = "gedi-subset"
_algo_id = "gedi-subset_ubuntu"
_version = "gedi-subset-0.2.4"
_queue = "maap-dps-worker-8gb"
_username = "<USERNAME>" # earthdata username
_aoi = "https://maap-ops-workspace.s3.amazonaws.com/shared/<USERNAME>/path/to/aoi.geojson"
_columns = "-"
_query = "-"
_limit = "-"

In [ ]:
# Submitting a DPS job
# Sets 'result_job_id' to use with maap.getJobResult
result = maap.submitJob(
    identifier=_identifier,
    algo_id=_algo_id,
    version=_version,
    queue=_queue,
    username=_username,
    aoi=_aoi,
    columns=_columns,
    query=_query,
    limit=_limit,
)
result_job_id = result["job_id"]
print(result)

In [ ]:
# Recursively call maap.getJobResult until a response 200 is received
# Sets 'location' to path in my-private-bucket in ADE
import backoff

@backoff.on_predicate(backoff.expo, max_value=60)
def get_result(result_job_id):
    result = maap.getJobResult(result_job_id)
    if (result.status_code == 200):
        location = result._content.decode("UTF-8").split("dps_output/")[1].split("</wps:Data>")[0]
        print(result._content.decode("UTF-8"))
        return True, location
    else:
        return False

bool, location = get_result(result_job_id)

In [ ]:
# (Optional Cell) Call to maap.getJobResult without using the Backoff library
result = maap.getJobResult(result_job_id)
print(result._content.decode("UTF-8"))

## 3. Verify the Results

Using the location set during the maap.getJobResult, plot the gpkg generated by the DPS job to view the results.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
gedi_gnq = gpd.read_file(f'~/my-private-bucket/dps_output/{location}/gedi_subset.gpkg')

In [ ]:
agbd_colors = plt.cm.get_cmap('viridis_r')
gedi_gnq.plot(column='agbd', cmap=agbd_colors);